# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [1]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [2]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,身長,しんちょう
1,隊,たい
2,繋ぐ,つなぐ
3,仏,ほとけ
4,現実,げんじつ


#### load Models

In [3]:
# azure_llm = AzureChatOpenAI(
#     azure_endpoint="https://tooldev-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
#     api_key=os.environ["AZURE_API_KEY"],
#     model_name="gpt-4o",
#     api_version="2025-01-01-preview",
#     temperature=0.5,
# )

In [4]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.5),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [5]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
問題5

つぎのことばの使い方として最もよいものを、1・2・3・4から一つえらびなさい。

31. 内容
	1.	修理のため、エアコンの内容を一度取り出します
	2.	鍋の中にカレーの内容を入れて、1時間くらい煮てください
	3.	古い財布から新しい財布へ内容を移しました
	4.	この手紙の内容は、ほかの人には秘密にしてください

32. 活動
	1.	彼は有名なロック歌手だったが、今は活動していない
	2.	山に登ると、新鮮な空気が活動していて気持ちがいい
	3.	さっきまで活動していたパソコンが、急に動かなくなった
	4.	駅前のコンビニは24時間活動しているので便利だ

33. 落ち着く
	1.	この辺りは、冬になると雪が落ち着いて、春になるまで溶けません
	2.	シャツにしみが落ち着いてしまって、洗ってもきれいになりません
	3.	あそこの木の上に美しい鳥が落ち着いています
	4.	大好きなこの曲を聞くと、いつも気持ちが落ち着きます

34. ぐっすり
	1.	遠慮しないで、ぐっすり食べてください
	2.	優勝できたのは、毎日ぐっすり練習したからだと思う
	3.	今日は疲れているので、朝までぐっすり眠れそうだ
	4.	古い友人と久しぶりに会って、ぐっすりおしゃべりした

35. 性格
	1.	日本の古い性格に興味があるので、神社やお寺によく行きます
	2.	森さんはおとなしい性格で、自分の意見はあまり言いません
	3.	値段が高くても、塗装で性格のいい車を買うつもりです
	4.	音楽の性格を伸ばすために、5歳から専門家の指導を受けました
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = aws_llm.invoke(f"""You are a Japanese teacher. Your job is to write 5 word usage for candidates from the following sentences, select the most appropriate usage of the word from options 1, 2, 3, and 4. (Includes Japanese idiomatic expressions and fixed collocations.)
"""
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            身長   しんちょう
1             隊      たい
2            繋ぐ     つなぐ
3             仏     ほとけ
4            現実    げんじつ
...         ...     ...
2134         切れ      きれ
2135         討つ      うつ
2136         場面     ばめん
2137         変化     へんか
2138         警告    けいこく

[2139 rows x 2 columns]


In [6]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [7]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で商品の特徴や価格を尋ねる | 電車やバスの乗り方を尋ねる | 今日の天気や気温について話す| 映画やドラマの感想を話す | 仕事のスケジュールや業務内容を話す"})
display(Markdown(kanji["question"]))

Error raised by bedrock service
Traceback (most recent call last):
  File "C:\Users\Administrator\PycharmProjects\pythonProject\jlpt\venv\Lib\site-packages\langchain_aws\llms\bedrock.py", line 956, in _prepare_input_and_invoke
    response = self.client.invoke_model(**request_options)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\PycharmProjects\pythonProject\jlpt\venv\Lib\site-packages\botocore\client.py", line 570, in _api_call
    return self._make_api_call(operation_name, kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\PycharmProjects\pythonProject\jlpt\venv\Lib\site-packages\botocore\context.py", line 124, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\PycharmProjects\pythonProject\jlpt\venv\Lib\site-packages\botocore\client.py", line 1031, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.exceptions.

ClientError: An error occurred (UnrecognizedClientException) when calling the InvokeModel operation: The security token included in the request is invalid.